<a href="https://colab.research.google.com/github/ErisonBarros/Basico_Python.github.io/blob/erison.barros/Aulas_Geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Utilizando Python para verificar coordenadas geográficas em um mapa KML

Muitas vezes deseja-se verificar se uma localização geográfica pertence a um determinado mapa, cuja representação é um arquivo no formato KML (Keyhole Markup Language). Este é um formato criado para armazenar dados geográficos e mapas.

https://egmara-santos.medium.com/utilizando-python-para-verificar-coordenadas-geogr%C3%A1ficas-em-um-mapa-kml-c4d61541e5ce

# Instalação das Bibliotecas



In [14]:
pip install geopandas


     |████████████████████████████████| 994 kB 7.3 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 15.8 MB/s 


In [15]:
pip install shapely

In [16]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd

# Lendo o dataset e o KML

Suponha que você possua um arquivo KML que represente um mapa, chamado de ‘meu_mapa.kml’, e um arquivo CSV de localizações geográficas de latitude e longitude, chamado de ‘localizacoes.csv’.

Utilize o GeoPandas para ler o arquivo KML e atribua o resultado a uma variável

In [59]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
meu_mapa = gpd.read_file('/content/Numeração Ordenada.kml', driver='KML')
meu_mapa

,Name,Description,geometry
0,P1,,POINT Z (-36.44497 -8.91909 0.00000)
1,P2,,POINT Z (-36.44365 -8.92187 0.00000)
2,P3,,POINT Z (-36.44459 -8.92202 0.00000)
3,P4,,POINT Z (-36.44640 -8.91960 0.00000)


A variável meu_mapa é um GeoDataFrame e a coluna ‘geometry’ contém a forma geométrica que representa o mapa KML, um polígono.

In [18]:
type(meu_mapa)

geopandas.geodataframe.GeoDataFrame

In [35]:
localizacoes = pd.read_csv('/content/localização3.csv',delimiter=';')
localizacoes

,Name,desc,long,lat
0,0,P1,-36.44497,-8.91909
1,1,P2,-36.44365,-8.92187
2,2,P3,-36.44459,-8.92202
3,3,P4,-36.44640,-8.91960


O bom e velho tratamento dos dados
Está quase tudo pronto para você realizar a checagem das localizações no mapa disponibilizado! Para que isso seja possível, você precisa converter cada uma dessas localizações em uma variável do tipo Point:

In [90]:
pontos_geometricos = []
for xy in zip(localizacoes['long'], localizacoes['lat']):
    pontos_geometricos.append(Point(xy))

e incluir esta lista em uma coluna de um GeoDataFrame para as localizações:

In [40]:
localizacoes_geo = gpd.GeoDataFrame(localizacoes)
                                    
localizacoes_geo

,Name,desc,long,lat
0,0,P1,-36.44497,-8.91909
1,1,P2,-36.44365,-8.92187
2,2,P3,-36.44459,-8.92202
3,3,P4,-36.44640,-8.91960


In [96]:
for i in range(0, len(localizacoes_geo)):
    ponto = localizacoes_geo.geometry.loc[i]
    check = meu_mapa.contains(ponto).values
    print(check)
    localizacoes_geo["pertence_ao_mapa"].loc[i] = check
localizacoes_geo.head(5)

[False False False False]


KeyError: ignored

O bom e velho tratamento dos dados
Está quase tudo pronto para você realizar a checagem das localizações no mapa disponibilizado! Para que isso seja possível, você precisa converter cada uma dessas localizações em uma variável do tipo Point:

In [86]:
pontos_geometricos = []
for xy in zip(localizacoes['long'], localizacoes['lat']):
    pontos_geometricos.append(Point(xy))

e incluir esta lista em uma coluna de um GeoDataFrame para as localizações:

In [87]:
localizacoes_geo = gpd.GeoDataFrame(localizacoes,
                                    crs = {'init': 'epsg:4326'},
                                    geometry = pontos_geometricos)
localizacoes_geo

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Name,desc,long,lat,geometry
0,0,P1,-36.44497,-8.91909,POINT (-36.44497 -8.91909)
1,1,P2,-36.44365,-8.92187,POINT (-36.44365 -8.92187)
2,2,P3,-36.44459,-8.92202,POINT (-36.44459 -8.92202)
3,3,P4,-36.44640,-8.91960,POINT (-36.44640 -8.91960)


**Verificando as localizações no polígono!**
Para verificar quais localizações do conjunto de dados pertencem a região do polígono representado pela variável meu_mapa (mapa KML), o método contains() possui a solução para isto! Este retorna ‘True’ caso um ponto do tipo Point pertença a região e ‘False’, caso contrário. Utilize este método e crie uma nova coluna booleana chamada no GeoDataFrame:

In [85]:
for i in range(0, len(localizacoes_geo)):
    ponto = localizacoes_geo.geometry.loc[i]
    check = meu_mapa.contains(ponto).values
    localizacoes_geo["pertence_ao_mapa"].loc[i] = check
localizacoes_geo.head(4)

,Name,desc,long,lat,geometry
0,0,P1,-36.44497,-8.91909,POINT (-36.44497 -8.91909)
1,1,P2,-36.44365,-8.92187,POINT (-36.44365 -8.92187)
2,2,P3,-36.44459,-8.92202,POINT (-36.44459 -8.92202)
3,3,P4,-36.44640,-8.91960,POINT (-36.44640 -8.91960)


In [83]:
for i in range(0,len(localizacoes_geo)):
  ponto = localizacoes_geo.geometry.loc[i]
  check = meu_mapa.contains(ponto).values
  localizacoes_geo["pertence_ao_mapa"].loc[i] = check
  localizacoes_geo.head(4)
print(i)
print(ponto)
print(check)



KeyError: ignored

In [68]:
check = meu_mapa.contains(ponto).values
check

array([False, False, False, False])

Visualizando o KML e o dataset com o Folium!
Concluímos anteriormente que somente uma localização não pertence a região KML. Vamos visualizar esse dado?
O Folium é uma biblioteca bastante útil para a visualização de mapas em formato GeoJSON. Podemos converter o nosso arquivo KML, ‘meu_mapa.kml’, em um GeoJSON, ‘meu_mapa.geojson’ usando este link. Seguindo alguns passos deste belo tutorial aqui do Medium, você consegue visualizar o mapa e o dataset!
Importe a biblioteca de visualização Folium e defina uma figura:

In [55]:
print("Pontos dentro da região KML: ", sum(localizacoes_geo.pertence_ao_mapa == True))
print("Pontos fora da região KML: ", sum(localizacoes_geo.pertence_ao_mapa == False))

AttributeError: ignored

[GeoJson](https://mygeodata.cloud/converter/kml-to-geojson)

```
# Isto está formatado como código
```



In [42]:
import folium
fig = folium.Map(width=900, height=600)

In [ ]:
geo_json_mapa = json.load(open('/content/limite.geojson'))
folium.Choropleth(
    geo_data = geo_json_mapa,
    fill_color = "steelblue",
    fill_opacity = 0.4,
    line_color = "steelblue",
    line_opacity = 0.9
).add_to(fig)

In [58]:
pontos = localizacoes_geo[['latitude', 'longitude']].values.tolist()
for point in range(0, len(pontos)):
    folium.Marker(locationlist[point]).add_to(fig)
fig

KeyError: ignored